In [2]:
import pandas as pd
import numpy as np

In [7]:
# 파일 불러오기
Extract_result = pd.read_csv('../20230210_QC_Cycle_2/Extract_result.txt', sep='\t', engine='python', encoding='cp949', names=['variants', 'SNP', 'major','minor'])

In [11]:
plink_lmiss = pd.read_csv('../20230210_QC_Cycle_2/plink.lmiss', sep='\t', engine='python', encoding='cp949')

In [8]:
Extract_result

,variants,SNP,major,minor
0,1:956852,kgp2263728,C,T
1,1:956852,1:956852,C,T
2,1:1142150,kgp9143777,A,G
3,1:1142150,1:1142150,A,G
4,1:1847886,exm4638,T,C
...,...,...,...,...
5424,22:50335034,22:50335034,G,A
5425,22:50346042,rs56091001,T,C
5426,22:50346042,22:50346042,T,C
5427,22:51048400,rs2269380,G,A


In [12]:
plink_lmiss

,CHR,SNP,N_MISS,N_GENO,F_MISS
0,1,kgp2263728,2,2119,0.000944
1,1,1:956852,7,2119,0.003303
2,1,kgp9143777,0,2119,0.000000
3,1,1:1142150,2,2119,0.000944
4,1,exm4638,3,2119,0.001416
...,...,...,...,...,...
5424,22,22:50335034,2,2119,0.000944
5425,22,rs56091001,7,2119,0.003303
5426,22,22:50346042,7,2119,0.003303
5427,22,rs2269380,19,2119,0.008966


In [13]:
plink_lmiss[['F_MISS']]

,F_MISS
0,0.000944
1,0.003303
2,0.000000
3,0.000944
4,0.001416
...,...
5424,0.000944
5425,0.003303
5426,0.003303
5427,0.008966


In [20]:
plink_lmiss['F_MISS'].dtype

dtype('float64')

In [21]:
plink_lmiss[plink_lmiss['F_MISS'] < 0]

,CHR,SNP,N_MISS,N_GENO,F_MISS


In [14]:
# 데이터 머지
merge = pd.merge(Extract_result, plink_lmiss, how='outer', on='SNP')
merge

,variants,SNP,major,minor,CHR,N_MISS,N_GENO,F_MISS
0,1:956852,kgp2263728,C,T,1,2,2119,0.000944
1,1:956852,1:956852,C,T,1,7,2119,0.003303
2,1:1142150,kgp9143777,A,G,1,0,2119,0.000000
3,1:1142150,1:1142150,A,G,1,2,2119,0.000944
4,1:1847886,exm4638,T,C,1,3,2119,0.001416
...,...,...,...,...,...,...,...,...
5424,22:50335034,22:50335034,G,A,22,2,2119,0.000944
5425,22:50346042,rs56091001,T,C,22,7,2119,0.003303
5426,22:50346042,22:50346042,T,C,22,7,2119,0.003303
5427,22:51048400,rs2269380,G,A,22,19,2119,0.008966


In [15]:
df = merge[['variants', 'SNP','F_MISS']]

# check 컬럼 생성
df['check'] = 0

df

C:\Users\user\AppData\Local\Temp\ipykernel_3676\1797800720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['check'] = 0


,variants,SNP,F_MISS,check
0,1:956852,kgp2263728,0.000944,0
1,1:956852,1:956852,0.003303,0
2,1:1142150,kgp9143777,0.000000,0
3,1:1142150,1:1142150,0.000944,0
4,1:1847886,exm4638,0.001416,0
...,...,...,...,...
5424,22:50335034,22:50335034,0.000944,0
5425,22:50346042,rs56091001,0.003303,0
5426,22:50346042,22:50346042,0.003303,0
5427,22:51048400,rs2269380,0.008966,0


### 같은 값 3개 데이터 분리

#### .value_counts()를 데이터 프레임화

.value_counts().to_frame()

.value_counts()를 데이터 프레임화 시킴

#### 2차원 -> 1차원
1. np.concatenate(list).tolist()
2. np.array(list).flatten().tolist()

In [16]:
var_vlucnt = df['variants'].value_counts().to_frame()
var_vlucnt

,variants
3:47061183,3
6:32280774,3
6:32431732,3
1:956852,2
11:2222002,2
...,...
6:5637740,2
6:5655467,2
6:5679414,2
6:6310536,2


In [17]:
# var_vlucnt의 variants 컬럼의 값이 3인 인덱스를 불러옴
var_cnt = list(var_vlucnt[var_vlucnt['variants'] == 3].index)
var_cnt
three_idx = []

# 위의 인덱스가 df variants	컬럼의 값임
# 값이 3개가 같은 df인덱스를 불러옴
for i in range(len(var_cnt)):
    three_idx.append(list(df[df['variants'] == var_cnt[i]].index))

# 2차원 -> 1차원 
three_idx = np.array(three_idx).flatten().tolist()
three_idx


[982, 983, 984, 1959, 1960, 1961, 1982, 1983, 1984]

In [18]:
# 위의 구한 인덱스의 값을 가져온다. 
df.loc[three_idx, 'check'] = 3
df[df['check'] == 3]

,variants,SNP,F_MISS,check
982,3:47061183,rs2290547.1,0.006135,3
983,3:47061183,rs2290547.2,0.015100,3
984,3:47061183,rs2290547,0.004719,3
1959,6:32280774,JHU_6.32280773,0.000472,3
1960,6:32280774,6:32280774,0.000472,3
1961,6:32280774,rs117627796,0.000944,3
1982,6:32431732,JHU_6.32431731,0.005663,3
1983,6:32431732,6:32431732,0.005663,3
1984,6:32431732,rs59406271,0.005663,3


In [19]:
df3 = df[df['check'] == 3]
df3

,variants,SNP,F_MISS,check
982,3:47061183,rs2290547.1,0.006135,3
983,3:47061183,rs2290547.2,0.015100,3
984,3:47061183,rs2290547,0.004719,3
1959,6:32280774,JHU_6.32280773,0.000472,3
1960,6:32280774,6:32280774,0.000472,3
1961,6:32280774,rs117627796,0.000944,3
1982,6:32431732,JHU_6.32431731,0.005663,3
1983,6:32431732,6:32431732,0.005663,3
1984,6:32431732,rs59406271,0.005663,3


In [20]:
df2 = df[df['check'] == 0]
df2

,variants,SNP,F_MISS,check
0,1:956852,kgp2263728,0.000944,0
1,1:956852,1:956852,0.003303,0
2,1:1142150,kgp9143777,0.000000,0
3,1:1142150,1:1142150,0.000944,0
4,1:1847886,exm4638,0.001416,0
...,...,...,...,...
5424,22:50335034,22:50335034,0.000944,0
5425,22:50346042,rs56091001,0.003303,0
5426,22:50346042,22:50346042,0.003303,0
5427,22:51048400,rs2269380,0.008966,0


### 데이터 중복 2개 처리

In [21]:
# df1.groupby('variants').agg('max')
df2_1 = df2.groupby('variants').agg('max')
df2 = df2_1.reset_index()
df2

,variants,SNP,F_MISS,check
0,10:103368654,rs7006,0.001416,0
1,10:105218252,kgp281354,0.008023,0
2,10:10527345,rs1408381,0.000472,0
3,10:108778483,rs10884400,0.000472,0
4,10:108781532,rs11193170,0.001416,0
...,...,...,...,...
2705,9:93563536,rs290986,0.001888,0
2706,9:94822540,rs7872515,0.015100,0
2707,9:97314741,rs16910061,0.001888,0
2708,9:9924724,rs7849581,0.000944,0


In [22]:
# 중복 데이터 없음 확인
a = []
for i in df2.value_counts():
    if i != 1:
        a.append(i)

print(a)

[]


In [23]:
df2[df2['variants'] == '1:956852']

,variants,SNP,F_MISS,check
1202,1:956852,kgp2263728,0.003303,0


### 데이터 중복 3개 처리

In [24]:
df3_1 = df3.reset_index()
df3_1.groupby('variants').agg('min')#.reset_index()

,index,SNP,F_MISS,check
variants,,,,
3:47061183,982,rs2290547,0.004719,3
6:32280774,1959,6:32280774,0.000472,3
6:32431732,1982,6:32431732,0.005663,3


In [25]:
df3_1.groupby('variants').agg('min')['index'].values

array([ 982, 1959, 1982], dtype=int64)

In [26]:
df3

,variants,SNP,F_MISS,check
982,3:47061183,rs2290547.1,0.006135,3
983,3:47061183,rs2290547.2,0.015100,3
984,3:47061183,rs2290547,0.004719,3
1959,6:32280774,JHU_6.32280773,0.000472,3
1960,6:32280774,6:32280774,0.000472,3
1961,6:32280774,rs117627796,0.000944,3
1982,6:32431732,JHU_6.32431731,0.005663,3
1983,6:32431732,6:32431732,0.005663,3
1984,6:32431732,rs59406271,0.005663,3


In [35]:
df3.loc[df3_1.groupby('variants').agg('min')['index'].values,'check'] = 4
df3

,variants,SNP,F_MISS,check
529,1:17349179,rs587782604,0.001902,4
530,1:17349179,rs587782604.1,0.002378,3
531,1:17349179,rs587782604.2,0.001427,3
536,1:17371377,rs74315369,0.001427,4
537,1:17371377,rs74315369.1,0.000476,3
...,...,...,...,...
17888,26:14502,rs368007903,0.000476,3
17889,26:14502,200610-140,0.001427,3
17914,26:15758,exm-rs41337244,0.000951,4
17915,26:15758,MTReverseCYB_35,0.000951,3


In [36]:
df3 = df3[df3['check'] == 3]
df3

,variants,SNP,F_MISS,check
530,1:17349179,rs587782604.1,0.002378,3
531,1:17349179,rs587782604.2,0.001427,3
537,1:17371377,rs74315369.1,0.000476,3
538,1:17371377,rs74315369.2,0.001902,3
568,1:21551817,rs367812436.2,0.000000,3
...,...,...,...,...
17882,26:14178,MitoT14179C,0.000476,3
17888,26:14502,rs368007903,0.000476,3
17889,26:14502,200610-140,0.001427,3
17915,26:15758,MTReverseCYB_35,0.000951,3


### 데이터 합치기 & 출력

In [37]:
df_concat = pd.concat([df2, df3])

In [38]:
df_concat

,variants,SNP,F_MISS,check
0,0:0,rs886044784,1.000000,0
1,10:100150819,Variant1012,0.003804,0
2,10:100995441,rs201180054.2,0.000951,0
3,10:101560288,rs777902199.2,0.000476,0
4,10:101578577,rs56199535.3,0.001427,0
...,...,...,...,...
17882,26:14178,MitoT14179C,0.000476,3
17888,26:14502,rs368007903,0.000476,3
17889,26:14502,200610-140,0.001427,3
17915,26:15758,MTReverseCYB_35,0.000951,3


In [40]:
df_f = df_concat[['SNP']]
df_f

,SNP
0,rs886044784
1,Variant1012
2,rs201180054.2
3,rs777902199.2
4,rs56199535.3
...,...
17882,MitoT14179C
17888,rs368007903
17889,200610-140
17915,MTReverseCYB_35


In [41]:
df_f = df_f.rename(columns=df_f.loc[0])
df_f = df_f[1:]
df_f

,rs886044784
1,Variant1012
2,rs201180054.2
3,rs777902199.2
4,rs56199535.3
5,rs56220353.2
...,...
17882,MitoT14179C
17888,rs368007903
17889,200610-140
17915,MTReverseCYB_35


In [45]:
# df_f.to_csv('./duplicated_SNPs_test.txt', sep='\t', index=False)
df_f.to_csv('../20230217_Cal_QC_Cycle_1/duplicated_SNPs.txt', sep='\t', index=False)

출력값 중에 0이 존재함

In [92]:
df[df['variants'] == '10:53358307']

,variants,SNP,F_MISS,check
3359,10:53358307,kgp39715616,0.0,0
3360,10:53358307,kgp8489311,0.0,0


In [93]:
zore2 = df2[df2['F_MISS'] == 0]
zore2

,variants,SNP,F_MISS,check
5,10:108788088,rs11193173,0.0,0
73,10:53358307,kgp8489311,0.0,0
80,10:59743916,JHU_10.59743915,0.0,0
87,10:59826399,kgp11412263,0.0,0
138,10:82474197,rs2075374,0.0,0
...,...,...,...,...
2578,8:23492936,rs12548733,0.0,0
2588,8:25831778,rs13256025,0.0,0
2592,8:2633080,rs117502124,0.0,0
2655,9:136029301,rs75765336,0.0,0


In [95]:
zore1 = df[df['F_MISS'] == 0]
zore1 = zore1.rename(columns={'F_MISS':'F_MISS1'})
zore1

,variants,SNP,F_MISS1,check
2,1:1142150,kgp9143777,0.0,0
22,1:2444470,imm_1_2434330,0.0,0
31,1:2465671,1:2465671,0.0,0
32,1:2466633,imm_1_2456493,0.0,0
35,1:2491306,kgp10382546,0.0,0
...,...,...,...,...
5357,22:20164589,rs385773,0.0,0
5364,22:28150109,kgp6055772,0.0,0
5390,22:37593080,22:37593080,0.0,0
5394,22:39732776,kgp4804636,0.0,0


In [96]:
# F_MISS이 null 값이 아닌 값을 찾아야함 나온 컬럼이 142개여야 함
zore = pd.merge(zore1, zore2, how='outer', on='variants')
zore0 = zore.dropna(axis=0)
zore0

,variants,SNP_x,F_MISS1,check_x,SNP_y,F_MISS,check_y
38,1:200877786,imm_1_199144409,0.0,0,imm_1_199144409,0.0,0.0
39,1:200877786,1:200877786,0.0,0,imm_1_199144409,0.0,0.0
45,1:235715661,rs387492,0.0,0,rs387492,0.0,0.0
46,1:235715661,kgp8103011,0.0,0,rs387492,0.0,0.0
61,2:30756196,rs829624,0.0,0,rs829624,0.0,0.0
...,...,...,...,...,...,...,...
722,19:14846335,kgp7047678,0.0,0,rs2732794,0.0,0.0
733,20:6647633,rs11696118,0.0,0,rs11696118,0.0,0.0
734,20:6647633,kgp9408193,0.0,0,rs11696118,0.0,0.0
740,20:49738465,rs230010,0.0,0,rs230010,0.0,0.0


In [ ]:
zore0[zore0['F_MISS1'] != 0]

,variants,F_MISS1,F_MISS


출력값 중에 0은 같은 variants이 모두 0을 가지고 있다.

In [ ]:
df2

,variants,F_MISS
0,10:103368654,0.001416
1,10:105218252,0.008023
2,10:10527345,0.000472
3,10:108778483,0.000472
4,10:108781532,0.001416
...,...,...
2708,9:93563536,0.001888
2709,9:94822540,0.015100
2710,9:97314741,0.001888
2711,9:9924724,0.000944


In [ ]:
# df2.to_csv('./duplicated_SNPs.csv')